## Second heuristic - Preliminary implementation

### Description

If there is a deposit and a withdraw transaction with **unique** gas prices (e.g., 3.1415926 Gwei), then we consider the deposit and the withdraw transactions linked. The corresponding deposit transaction can be removed from any other withdraw transaction’s anonymity set.

In [1]:
using DataFrames
using CSV
using ProgressBars

In [2]:
withdraw_transactions_df = CSV.read("../data/lighter_complete_withdraw_txs.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/lighter_complete_deposit_txs.csv", DataFrame)

non_relayer_withdraw_transactions_df = filter(row -> row.from_address == row.recipient_address, withdraw_transactions_df);

In [3]:
ENV["COLUMNS"]=10000
ENV["LINES"]=10;

### Function summary: filter_by_unique_gas_price

Filters a transaction DataFrame, leaving only the transactions (rows) that have unique gas_price within all transactions.

In [4]:
# Filters a transaction DataFrame, leaving only the rows that have unique gas_price.

function filter_by_unique_gas_price(transactions_df)
    unique_gas_prices = filter(row -> row.count==1, combine(groupby(transactions_df, :gas_price), nrow => :count))[!, "gas_price"]
    filter(row -> row.gas_price ∈ unique_gas_prices, transactions_df)
end

filter_by_unique_gas_price (generic function with 1 method)

### Function summary: filter_by_unique_gas_price_by_pool

Filters a transaction DataFrame, leaving only the transactions (rows) that have unique gas_price within all transactions of the same pool.

In [5]:
# Filters a transaction DataFrame, leaving only the rows that have unique gas_price.

function filter_by_unique_gas_price_by_pool(transactions_df)
    unique_gas_prices_per_pool = filter(row -> row.count == 1, combine(groupby(transactions_df, [:gas_price, :tornado_cash_address]), nrow => :count))[!, [:gas_price, :tornado_cash_address]]
    tuple_set = Set([(row.gas_price, row.tornado_cash_address) for row in eachrow(unique_gas_prices_per_pool)])
    
    filter(row -> (row.gas_price, row.tornado_cash_address) ∈ tuple_set, transactions_df)
end

filter_by_unique_gas_price_by_pool (generic function with 1 method)

In [6]:
@time filter_by_unique_gas_price_by_pool(deposit_transactions_df)

  3.150671 seconds (8.07 M allocations: 471.683 MiB, 3.97% gc time)


,Column1,Unnamed: 0,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address
,Int64,Int64,String,Int64,Int64,String,String,Float64,Int64,Int64,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Float64?,Float64?,Float64?,Int64,String
1,0,0,0xcf97c470a56d96625c7240d3004ae2abd9141d7ffc4383ab6f29a181e3562e8b,4,10,0xb050dec5a9010f8b77a3962369b7bc737d3ed4a5,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0.0,1200000,56000000000,1677906,1048832,missing,missing,1,2020-11-02 17:47:30 UTC,11179130,0x21d86cba454fea4f7e43c68763d4cffec101b614554635a3f15d538049463d4f,missing,missing,missing,56000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d
2,2,10,0x7baf0a76f35c1dece97fff883aa7174454bed460b1bade05844080017170fc6c,240,171,0x8c4c44fd06f7f98f08bf6a9ca156cec9ee1f31f8,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,0.0,800000,105000000000,10723996,800000,missing,missing,0,2021-01-06 19:04:40 UTC,11602841,0x7c5f21ea2a92f5182ce8648f152b6fb3b4379096309dc2ba232ac663c4a0d1d7,missing,missing,missing,105000000000,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144
3,4,12,0xbd83053f8afa7777f54a4aca6b8e112fa31b888922dc5b9a9a65eb66e9a6996f,7,63,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1.0e17,800000,30838446643,6222489,800000,missing,missing,0,2020-05-27 03:30:44 UTC,10145408,0x837b3482443f027f6f045644bf002243f72304686015a2d6265676b2a2fc630b,missing,missing,missing,30838446643,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
4,8,130,0xba7d56fea776705a937d912674cc56cf3ea71485c8fb6995d1c2ec00d44645fc,2,69,0x3a456bc9083bfe147719504aee8f296eb7355ee1,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f,0.0,1200000,100000000000,4634190,992258,missing,missing,1,2020-08-15 11:34:23 UTC,10664413,0x4075362cd93950767b2e5c3cd6765810b2973a2dd073186ed75141973fb1ae84,missing,missing,missing,100000000000,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f
5,10,132,0x6c416af65ea3a4bc096663c94f5b1fb0cba91607f61703657094f1f5441a3a12,3,51,0x27972d10f153099b3649ea8546a11d91315455e5,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f,0.0,1200000,71302125000,4108630,992258,missing,missing,1,2020-09-26 08:10:08 UTC,10937092,0x32f0f0fd04d3af8210d2eb956fdec21e09ebff5a55209c0a2c559bb1c034b158,missing,missing,missing,71302125000,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f
6,12,1549,0x830dbd534d13cd43cb078b7cad8a9c5137bb19aa8bf38e0c3b0e222b688d8340,0,63,0x43eefeb3db479e7b22e015572f38b6af633a43ff,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1.0e18,275000,595000000000,4390453,274947,missing,missing,0,2020-09-17 20:50:21 UTC,10881994,0x5270afd78906cc7264620b2c6fbf8c9221cf8053c8111edc0d7457966340dd81,missing,missing,missing,595000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936
7,17,1582,0x56837665efa6b3ed43685c79744f8b3c575090f7d076f7187c2ae3ace5aea0c9,8,41,0x0fcdae4a4aeb0497f39a209009dc200a6466b7bd,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,1.0e19,1200000,5231046931,5058769,978691,missing,missing,1,2020-01-24 05:54:15 UTC,9342713,0x2263281e65fb7d50125309a90864131650ca560ccd51c3e8e2847fbd2a18ee9f,missing,missing,missing,5231046931,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
8,67,1632,0x923c87764f1be67385a3ee07376b9c4a640c7a6402525337389654a35b8b57cd,205,13,0x67865c3eaf2e6a745a6b85cee169c7b74abe97d4,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,1.0e19,1200000,6500000000,4612395,978691,missing,missing,1,2020-02-28 03:00:12 UTC,9569564,0xd6b7228ba576816050a47c4c7f4c21b96d5fc384f3a68a67d5866cc59843a648,missing,missing,missing,6500000000,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
9,125,1690,0x038303d14327a2437bd789a70c0f7daa95b3ab89b46af0418fd0f677bdf2766c,82,163,0x3eb24c61590e6cd6fc26cf4fb938316206147859,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1.0e17,1200000,57151624549,9371665,978691,missing,missing,1,2020-10-16 09:18:01 UTC,11066064,0xf

In [7]:
unique_gas_price_deposits = filter_by_unique_gas_price(deposit_transactions_df)
first(unique_gas_price_deposits, 5)

,Column1,Unnamed: 0,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address
,Int64,Int64,String,Int64,Int64,String,String,Float64,Int64,Int64,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Float64?,Float64?,Float64?,Int64,String
1,4,12,0xbd83053f8afa7777f54a4aca6b8e112fa31b888922dc5b9a9a65eb66e9a6996f,7,63,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1.0e17,800000,30838446643,6222489,800000,missing,missing,0,2020-05-27 03:30:44 UTC,10145408,0x837b3482443f027f6f045644bf002243f72304686015a2d6265676b2a2fc630b,missing,missing,missing,30838446643,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
2,10,132,0x6c416af65ea3a4bc096663c94f5b1fb0cba91607f61703657094f1f5441a3a12,3,51,0x27972d10f153099b3649ea8546a11d91315455e5,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f,0.0,1200000,71302125000,4108630,992258,missing,missing,1,2020-09-26 08:10:08 UTC,10937092,0x32f0f0fd04d3af8210d2eb956fdec21e09ebff5a55209c0a2c559bb1c034b158,missing,missing,missing,71302125000,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f
3,12,1549,0x830dbd534d13cd43cb078b7cad8a9c5137bb19aa8bf38e0c3b0e222b688d8340,0,63,0x43eefeb3db479e7b22e015572f38b6af633a43ff,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1.0e18,275000,595000000000,4390453,274947,missing,missing,0,2020-09-17 20:50:21 UTC,10881994,0x5270afd78906cc7264620b2c6fbf8c9221cf8053c8111edc0d7457966340dd81,missing,missing,missing,595000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936
4,17,1582,0x56837665efa6b3ed43685c79744f8b3c575090f7d076f7187c2ae3ace5aea0c9,8,41,0x0fcdae4a4aeb0497f39a209009dc200a6466b7bd,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,1.0e19,1200000,5231046931,5058769,978691,missing,missing,1,2020-01-24 05:54:15 UTC,9342713,0x2263281e65fb7d50125309a90864131650ca560ccd51c3e8e2847fbd2a18ee9f,missing,missing,missing,5231046931,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
5,125,1690,0x038303d14327a2437bd789a70c0f7daa95b3ab89b46af0418fd0f677bdf2766c,82,163,0x3eb24c61590e6cd6fc26cf4fb938316206147859,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1.0e17,1200000,57151624549,9371665,978691,missing,missing,1,2020-10-16 09:18:01 UTC,11066064,0xfa752cf56aaec471375e2996736d20444b146199e2c3b84851cb16c65695cd3f,missing,missing,missing,57151624549,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc


### Function summary: same_gas_price_heuristic

This function receives a particular withdraw transaction and a DataFrame with the unique gas price deposits.

It returns a tuple:
* $(true, deposit$ $hash)$ when a deposit transaction with the same gas price as the withdrawal transaction is found.
* $(false, nothing)$ when such a deposit is not found.

In [8]:
# Given a withdrawal transaction and a DataFrame with unique gas_price deposit transactions, checks
# if there is a deposit transaction with the same gas_price as the withdrawal transaction.

function same_gas_price_heuristic(withdrawal_transaction, unique_gas_price_deposit_dataframe)
    for row in eachrow(unique_gas_price_deposit_dataframe)
        if (withdrawal_transaction.gas_price == row.gas_price) &&
            (withdrawal_transaction.block_timestamp > row.block_timestamp)
                return (true, row.hash)
        end
    end
    (false, nothing)
end

same_gas_price_heuristic (generic function with 1 method)

### Function summary: same_gas_price_heuristic_by_pool

This function receives a particular withdraw transaction and a DataFrame with the unique gas price deposits by pool.

It returns a tuple:
* $(true, deposit$ $hash)$ when a deposit transaction with the same gas price as the withdrawal transaction and from the same TCash pool is found.
* $(false, nothing)$ when such a deposit is not found.

In [9]:
# Given a withdrawal transaction and a DataFrame with unique gas_price deposit transactions, checks
# if there is a deposit transaction with the same gas_price as the withdrawal transaction.

function same_gas_price_heuristic_by_pool(withdrawal_transaction, unique_gas_price_by_pool_deposit_dataframe)
    for row in eachrow(unique_gas_price_by_pool_deposit_dataframe)
        if (withdrawal_transaction.gas_price == row.gas_price) &&
            (withdrawal_transaction.block_timestamp > row.block_timestamp) &&
            (withdrawal_transaction.tornado_cash_address == row.tornado_cash_address)
                return (true, row.hash)
        end
    end
    (false, nothing)
end

same_gas_price_heuristic_by_pool (generic function with 1 method)

In [10]:
same_gas_price_heuristic(withdraw_transactions_df[1,:], unique_gas_price_deposits)

(false, nothing)

### Function summary: apply_same_gas_price_heuristic

Applies the heuristic to all the withdraw_transactions DataFrame. Returns a dicionary mapping linked withdrawal and deposit transaction hashes.

In [11]:
# Applies the function to detect same unique gas_prices to all the withdraw_transactions data.
# Returns a list of tuples, each tuple with the deposit transaction hash in the first index and
# the withdrawal transaction hash in the second index.

function apply_same_gas_price_heuristic(deposit_dataframe, withdraw_dataframe)
    unique_gas_price_deposits = filter_by_unique_gas_price(deposit_dataframe)
    
    withdrawal_to_deposit = Dict()
    
    for withdraw_row in ProgressBar(eachrow(withdraw_dataframe), printing_delay=3)
        same_gas_deposit_hash = same_gas_price_heuristic(withdraw_row, unique_gas_price_deposits)
        if same_gas_deposit_hash[1]
            withdrawal_to_deposit[withdraw_row.hash] = same_gas_deposit_hash[2]
        end
    end
    withdrawal_to_deposit
end 

apply_same_gas_price_heuristic (generic function with 1 method)

### Function summary: apply_same_gas_price_heuristic_by_pool

Applies the heuristic to all the withdraw_transactions DataFrame. Returns a dicionary mapping linked withdrawal and deposit transaction hashes.

In [12]:
# Applies the function to detect same unique gas_prices to all the withdraw_transactions data.
# Returns a list of tuples, each tuple with the deposit transaction hash in the first index and
# the withdrawal transaction hash in the second index.

function apply_same_gas_price_heuristic_by_pool(deposit_dataframe, withdraw_dataframe)
    unique_gas_price_deposits = filter_by_unique_gas_price_by_pool(deposit_dataframe)
    
    withdrawal_to_deposit = Dict()
    
    for withdraw_row in ProgressBar(eachrow(withdraw_dataframe), printing_delay=3)
        same_gas_deposit_hash = same_gas_price_heuristic_by_pool(withdraw_row, unique_gas_price_deposits)
        if same_gas_deposit_hash[1]
            withdrawal_to_deposit[withdraw_row.hash] = same_gas_deposit_hash[2]
        end
    end
    withdrawal_to_deposit
end 

apply_same_gas_price_heuristic_by_pool (generic function with 1 method)

In [13]:
@time linked_transactions = apply_same_gas_price_heuristic(deposit_transactions_df, non_relayer_withdraw_transactions_df);

0.0%┣                                      ┫ 0/17.0k [00:03<-14:-11:-51, -3s/it]
0.0%┣                                        ┫ 1/17.0k [00:03<Inf:Inf, InfGs/it]
10.1%┣████                                   ┫ 1.7k/17.0k [00:06<00:55, 278it/s]
20.1%┣███████▉                               ┫ 3.4k/17.0k [00:09<00:36, 373it/s]
30.2%┣███████████▊                           ┫ 5.1k/17.0k [00:12<00:28, 422it/s]
40.1%┣███████████████▋                       ┫ 6.8k/17.0k [00:15<00:23, 450it/s]
50.0%┣███████████████████▌                   ┫ 8.5k/17.0k [00:18<00:18, 469it/s]
60.2%┣██████████████████████▉               ┫ 10.3k/17.0k [00:21<00:14, 484it/s]
70.4%┣██████████████████████████▊           ┫ 12.0k/17.0k [00:24<00:10, 496it/s]
80.6%┣██████████████████████████████▋       ┫ 13.7k/17.0k [00:27<00:07, 505it/s]
90.8%┣██████████████████████████████████▌   ┫ 15.5k/17.0k [00:30<00:03, 513it/s]


 32.285103 seconds (998.22 M allocations: 15.035 GiB, 7.03% gc time)


100.0%┣█████████████████████████████████████┫ 17.0k/17.0k [00:33<00:00, 518it/s]


In [14]:
# A DataFrame with the linked transactions.
linked_transactions_df = DataFrame("withdraw_hash"=> collect(keys(linked_transactions)), "deposit_hash"=> collect(values(linked_transactions)))

,withdraw_hash,deposit_hash
,Any,Any
1,0x571a9c4a9d2a2b0386285e048073a8d48e995f2b5a3b5cd36734023f06e45a44,0x35a464d0e00fc398607b092f7b3186cd6c8ec0c82f6f4b6d9978c71221647671
2,0x99b09453c62a24b9b7f78c27b37d1c06c2b9a64285412bf7e73609746a00abc0,0x1cf1ce4d853c7f7df4c44634e45080e6c038dcd0c4e651920138477d225cdd72
3,0x062c1c415381703a6a00022bbb93ac541ca0092f84c8cc34658e4224da945bdd,0x42caf310513b6eed265b9cd7e260c0325282d8d209035188d7d74b3ef85f509f
4,0x02a1c2250c4dea47f8f38f262ea723ff54de15e8ff6f4de9b19938757800868f,0x04e9849765f104c250e6da696b16fd2be7f0eca3254038c0ba21049b061b3748
5,0x197b4399402095682fd725c4fa7ce23450f1ffb4bd651dbe4f992b231f61160e,0x988638a03002b0bdf7023fdc03c0e7f9fdb7a493a913dc5b5bfc7e5c3c26f3db
6,0x3e7734771217c28f9fc6d5b412ab35b4f8554b22fa5bb4d59789a981d15cde34,0x425066c1edecb08e72b81b7e32cb01e9637cbd0e8442bc4ee06d071ce2690340
7,0x0ee6d471ad8bb4217ed0e416e2c51af2d6e486e0f3fe5ac187e0488add7b2a60,0x1cf1ce4d853c7f7df4c44634e45080e6c038dcd0c4e651920138477d225cdd72
8,0x2d9649c3437d75adf66be1e2aa0a20858fafc4bfdc9d591a643ae8c074e6fc64,0xb3df9977fcf74b589dac668cb4fb360732f76edb72d1a779c7e4ef57265d89cd
9,0x523ec14086e2788e60f3ec20bf57450a41175dd4d76990acf675199a145471d4,0x7f21ef508a9bfafb030071da5f98db43981d7ea5405fd327a46dafdb6f10b4c0


In [15]:
@time linked_transactions_by_pool = apply_same_gas_price_heuristic_by_pool(deposit_transactions_df, non_relayer_withdraw_transactions_df);

0.0%┣                                      ┫ 0/17.0k [00:03<-14:-11:-51, -3s/it]
0.0%┣                                        ┫ 1/17.0k [00:03<Inf:Inf, InfGs/it]
8.6%┣███▌                                    ┫ 1.5k/17.0k [00:06<01:04, 244it/s]
17.2%┣██████▊                                ┫ 2.9k/17.0k [00:09<00:44, 324it/s]
25.7%┣██████████                             ┫ 4.4k/17.0k [00:12<00:35, 364it/s]
34.2%┣█████████████▍                         ┫ 5.8k/17.0k [00:15<00:29, 388it/s]
42.9%┣████████████████▊                      ┫ 7.3k/17.0k [00:18<00:24, 406it/s]
51.7%┣████████████████████▏                  ┫ 8.8k/17.0k [00:21<00:20, 418it/s]
60.3%┣███████████████████████               ┫ 10.3k/17.0k [00:24<00:16, 427it/s]
69.1%┣██████████████████████████▎           ┫ 11.8k/17.0k [00:27<00:12, 435it/s]
77.8%┣█████████████████████████████▋        ┫ 13.3k/17.0k [00:30<00:09, 441it/s]
86.4%┣████████████████████████████████▉     ┫ 14.7k/17.0k [00:33<00:05, 446it/s]
94.9%┣██████████████████████

 34.921075 seconds (1.15 G allocations: 17.155 GiB, 7.50% gc time, 0.12% compilation time)


100.0%┣█████████████████████████████████████┫ 17.0k/17.0k [00:38<00:00, 450it/s]


In [16]:
# A DataFrame with the linked transactions by pool.
linked_transactions_by_pool_df = DataFrame("withdraw_hash"=> collect(keys(linked_transactions_by_pool)), "deposit_hash"=> collect(values(linked_transactions_by_pool)))

,withdraw_hash,deposit_hash
,Any,Any
1,0x185124e22a1f913f3f71db70fc38d7ae56cd2e671bc08ebcd44d2de6f488cdbe,0xf72d23287b3bf7dfc53633e3be0f728d9d5068eadb927eb79045d31d34f6538d
2,0xc1b92522a23530412f24c0edfe62be443d8d0c3217562ca184505249e0235bba,0x936d0599d352096cdfcc1df76c3a027ce3ff2d835cd0b3030e7aaa688ce2fdcb
3,0x88fde9e123ac0a9b011428ffb713c424c8aa8c5980a23fb7d8f2ba157f50d29e,0x0e1295215085d3f9e4799e8fd2a70fba45ce15dad27e284b71a503a630f5f542
4,0x062c1c415381703a6a00022bbb93ac541ca0092f84c8cc34658e4224da945bdd,0x42caf310513b6eed265b9cd7e260c0325282d8d209035188d7d74b3ef85f509f
5,0x4ce400034e4118bb1f57613ab19bb3b4697e1a569abe34b50a4ee26d03af6bd0,0x536caff3cb304258ddd02281f9122b73e1a0e2f24a6e102e45ab293b2cdc14a1
6,0xc4b5fc604fbddc059d9f734a6cd215da7a8396c78f3f40191081e2307db135f4,0x9da5cc9fd17e1f8f75a921dc0974fb7cf1d24077b5332a9630ca62c21e1263b3
7,0x4c5682d081e3085062c5f59bd29da9b80870fac27c94db1d4b58c443507dabbe,0xac60db623d36ab867f39572d07972b46809a7c7ceec8c852255bbb3730e0c5bf
8,0x0fae022876b43a8ffbf966b3135d935db856203d9862b1f560b9f75c9c2c2158,0xe70dd917d26dea5e9a668b1b6b87fef1baf5cdc0367e72211512a28fa51f7b04
9,0x1e84a2a314747ef8617b8ae62d0aa21058c8eab8e311d9100f376e88f870f13b,0x3e7dd307c469ded667789467b1feb3c9b263301ef862b833c01791944bf2f6a2


In [17]:
# CSV.write("../data/linked_transactions_same_gas_price_heuristic_by_pool.csv", linked_transactions_by_pool_df)